In [1]:
import sys
import os
myDir = os.getcwd()
parentDir = os.path.abspath(os.path.join(myDir, os.pardir))
sys.path.append(parentDir)

from pathlib import Path
path = Path(parentDir)
a=str(path.parent.absolute())

sys.path.append(a)

In [2]:
from MetaX.utils.taxaFuncAnalyzer import TaxaFuncAnalyzer
from MetaX.utils.taxaFuncPloter.basic_plot import BasicPlot


In [3]:
df_path = './example_data/SW_TaxaFunc.tsv'
meta_path = './example_data/SW_Meta.tsv'
sw = TaxaFuncAnalyzer(df_path, meta_path)

original df shape: (3999, 229)
after remove all zero row: (3999, 229)


In [4]:
sw.set_func('Description')
sw.set_multi_tables(level='s')

sw.set_group('Person')

batch_list is not set, Batch effect removal did not perform.
transform_method is not set, data transform did not perform.
normalize_method is not set, data normalization did not perform.
Original data shape: (3999, 229)
Starting to set Function table...
Function number: 830
Starting to set Taxa table...
Taxa number: 340
Taxa-Function number: 1272
group is set to Person
 {'V5', 'V1', 'V2', 'V3', 'V4'}


In [27]:
from pyecharts import options as opts
from pyecharts.charts import Bar


class BarPlot_js:
    def __init__(self, tfobj):
        self.tfobj =  tfobj
    # plot intensity line for each sample
    # Example: plot_intensity_line(sw, func_name=func_name, taxon_name=taxon_name, fig_size=(30,20))
    def rename_taxa(self, df):
        if df.index.name == 'Taxon':
            index_list = [i.split('|')[-1] for i in df.index.tolist()]
            df.index = index_list
        elif 'd__Bacteria' in df.index.tolist()[0]:
            new_index_list = []
            for i in df.index.tolist():
                taxon = i.split(' <')[0].split('|')[-1]
                func = i.split(' <')[1][:-1]
                new_index = f'{taxon} <{func}>'
                new_index_list.append(new_index)
            df.index = new_index_list
        return df

    def plot_intensity_bar(self, taxon_name:str=None, groups:list = None, func_name:str=None, peptide_seq=None, width:int=1200, height:int=800, df= None, title:str=None, rename_taxa:bool=False):
        if df is None:
            df = self.tfobj.get_intensity_matrix(taxon_name=taxon_name, func_name=func_name, peptide_seq=peptide_seq, groups= groups)
            if df.empty:
                raise ValueError('No data to plot')
        # rename taxa
        if rename_taxa:
            df = self.rename_taxa(df)
        #rename columns (sample name)
        col_names = df.columns.tolist()
        meta_df = self.tfobj.meta_df
        meta_name = self.tfobj.meta_name
        groups_list = []
        new_col_names = []
        for i in col_names:
            group = meta_df[meta_df['Sample'] == i]
            group = group[meta_name].values[0]
            new_col_names.append(f'{i} ({group})')
            groups_list.append(group)
        df.columns = new_col_names
        
        # create title
        if title is None:
            if taxon_name is None:
                title = f'{func_name}'
            elif func_name is None:
                title = f'{taxon_name}'
            elif peptide_seq is not None:
                title = f'The intensity of {peptide_seq}'
            elif taxon_name is None and func_name is None:
                title = 'The intensity of '
            else:
                title = f'{taxon_name}\n{func_name}'
            
            
        c = (
            Bar(init_opts=opts.InitOpts(width=f"{width}px", height=f"{height}px"))
            .add_xaxis(list(df.columns))
        )


        for name in df.index:
            c.add_yaxis(name, list(df.loc[name, :]), stack="stack1", category_gap="50%")
            
            c.set_series_opts(label_opts=opts.LabelOpts(is_show=False))
                            
            
        c.set_global_opts(
            datazoom_opts=[opts.DataZoomOpts( type_="inside", range_start=0, range_end=100,)],
                        legend_opts=opts.LegendOpts(pos_left="right", orient="vertical", pos_top="5%",),
                        toolbox_opts=opts.ToolboxOpts( is_show=True, orient="vertical", pos_left="right", pos_top="bottom"),
                        title_opts=opts.TitleOpts(title=f"{title}", pos_left="center" ),
                        xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=45)))

        return c
    
pic = BarPlot_js(sw).plot_intensity_bar(taxon_name='d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium infantis')

# pic.render_notebook()

In [6]:
sw.taxa_df.head(10)

,V1_01,V1_02,V1_03,V1_04,V1_05,V1_06,V1_07,V1_08,V1_09,V1_10,...,V5_29,V5_30,V5_31,V5_32,V5_33,V5_34,V5_35,V5_36,V5_37,V5_38
Taxon,,,,,,,,,,,,,,,,,,,,,
d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium adolescentis,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium breve,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium infantis,1037814,580417,1199873,592779,1921826,473415,0,763330,832110,778045,...,2725270,1884194,2448213,1971572,4471248,425820,303871,1749583,4044545,239669
d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium pseudocatenulatum,3320249,3195393,3514170,4277385,3837359,1864044,2802231,3456020,2398460,1879147,...,0,2449732,903497,11902,1598319,2276211,1492873,1287703,3339219,2391731
d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium ruminantium,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium scardovii,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,358922,0,0,0,0,0
d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Mycobacteriales|f__Mycobacteriaceae|g__Corynebacterium|s__Corynebacterium aurimucosum_E,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Propionibacteriales|f__Propionibacteriaceae|g__Cutibacterium|s__Cutibacterium acnes,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2477175,0,0,0,0
d__Bacteria|p__Actinobacteriota|c__Coriobacteriia|o__Coriobacteriales|f__Coriobacteriaceae|g__Collinsella|s__Collinsella sp002232035,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
